In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
from ansys.mapdl.core import launch_mapdl
from ansys.mapdl import reader as pymapdl_reader

In [8]:
import pyvista
# 기본적으로 plot이 안돼서 html로 plot하도록 만듦
pyvista.set_jupyter_backend('html')
pyvista.start_xvfb()

In [2]:
# Start an MAPDL instance
try:
    mapdl = launch_mapdl(start_instance=False, port=50052)
except:
    mapdl = launch_mapdl(start_instance=True, port=50052)

In [3]:
mapdl.finish()
mapdl.clear()
mapdl.prep7()

*** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2024 R2          24.2     ***
 Ansys Mechanical Enterprise Academic Research     
 01078600  VERSION=LINUX x64     15:23:39  NOV 25, 2024 CP=      1.180

                                                                               



          ***** MAPDL ANALYSIS DEFINITION (PREP7) *****

## Sine Kagome Lattice Creation

In [4]:
design_width = 60/1000
design_height = 25/1000
total_width = 144/1000
total_z = 5/1000
# density = 0.5

n_cell = 5
relative_density = 0.5

t_over_l = np.sqrt(3)/2 - np.sqrt(3 - 4*relative_density)/2
cell_size = design_height/(np.sqrt(3)*n_cell + t_over_l)
wall_thickness = cell_size*t_over_l

external_wall = True
external_wall_thickness = wall_thickness

In [5]:
side_length = cell_size - 1/np.sqrt(3)*wall_thickness
design_part = mapdl.rectng(-1/1000, design_width, 0, design_height)

In [6]:
for i in range(n_cell):
    j = 0
    while True:
        center = [cell_size/2 - np.sqrt(3)/2*wall_thickness + 2*j*cell_size - (i%2)*cell_size,
                  (cell_size + 1/np.sqrt(3)*wall_thickness)*np.sqrt(3)/2 + i*np.sqrt(3)*cell_size]
        # 조건문 넣을 필요 없이, design part 늘리면 해결되긴 함
        if center[0] - side_length < design_width:
            k_list = []
            for k in range(6):
                k_list.append(mapdl.k(x = center[0] + side_length*np.cos(np.pi/3*k), y = center[1] + side_length*np.sin(np.pi/3*k)))
            hole = mapdl.a(*k_list)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except:
                print(f"error2 at i={i}, j={j}")
        # 조건문 넣을 필요 없이, design part 늘리면 해결되긴 함
        if cell_size + 2*j*cell_size - (i%2)*cell_size < design_width:
            k1 = mapdl.k(x = cell_size + 2*j*cell_size - (i%2)*cell_size, y = wall_thickness + i*np.sqrt(3)*cell_size)
            k2 = mapdl.k(x = 2*cell_size - np.sqrt(3)*wall_thickness + 2*j*cell_size - (i%2)*cell_size, y = wall_thickness + i*np.sqrt(3)*cell_size)
            k3 = mapdl.k(x = (3/2)*cell_size - 1/2*np.sqrt(3)*wall_thickness + 2*j*cell_size - (i%2)*cell_size, y = np.sqrt(3)/2*cell_size - 1/2*wall_thickness + i*np.sqrt(3)*cell_size)
            hole = mapdl.a(k1, k2, k3)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except Exception as e:
                print(f"error2 at i={i}, j={j}")
                print(e)

            k1 = mapdl.k(x = cell_size + 2*j*cell_size - (i%2)*cell_size, y = np.sqrt(3)*cell_size + i*np.sqrt(3)*cell_size)
            k2 = mapdl.k(x = 2*cell_size - np.sqrt(3)*wall_thickness + 2*j*cell_size - (i%2)*cell_size, y = np.sqrt(3)*cell_size + i*np.sqrt(3)*cell_size)
            k3 = mapdl.k(x = (3/2)*cell_size - 1/2*np.sqrt(3)*wall_thickness + 2*j*cell_size - (i%2)*cell_size, y = np.sqrt(3)/2*cell_size + 3/2*wall_thickness + i*np.sqrt(3)*cell_size)
            hole = mapdl.a(k1, k2, k3)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except Exception as e:
                print(f"error3 at i={i}, j={j}")
                print(e)

        if cell_size/2 - 1/np.sqrt(3)*wall_thickness + 2*j*cell_size > design_width:
            break

        j += 1

In [ ]:
# sine kagome: 점 여러개 찍어서 sine 곡선 생성 + 그 곡선을 따라서 원 움직이기 + 이후에 design width 바깥쪽에 있는 직사각형 더하기
mapdl.adrag


In [7]:
mapdl.rectng(x1=-total_width/2+design_width/2, x2=0, y1=0, y2=design_height)
mapdl.rectng(design_width, total_width/2+design_width/2, 0, design_height)
mapdl.aadd("all")

4

In [10]:
mapdl.vext("ALL", dz = total_z)

EXTRUDE ALL AREAS 
      IN DIRECTION   0.000000    ,  0.000000    , 0.5000000E-02
      SCALED         0.000000    ,  0.000000    ,  0.000000

## Preprocessing

In [16]:
stress_strain = pd.read_excel('PLA stress-strain.xlsx')
stress_strain = stress_strain.iloc[:8]  # due to limitations of MISO model
stress_strain['y'] = stress_strain['y (MPa)']*1e6
stress_strain['x'] = stress_strain['x']/100
UTS = stress_strain['y'].max()
Youngs_Modulus = stress_strain['y'][1]/stress_strain['x'][1]
max_plastic_strain = stress_strain['x'].iloc[-1]

mapdl.mp('EX', 1, Youngs_Modulus)   # Young's Modulus in Pascals
mapdl.mp('PRXY', 1, 0.33)   # Poisson's Ratio

# Non linear 부분 정의
stress_strain['x'] = stress_strain['x'] - stress_strain['y']/Youngs_Modulus
mapdl.tb('PLASTIC', 1, '', 'MISO')
for i in range(len(stress_strain)):
    mapdl.tbpt(i, stress_strain['x'][i], stress_strain['y'][i])

In [17]:
mapdl.allsel()

SELECT ALL ENTITIES OF TYPE= ALL  AND BELOW

## Simulation

In [ ]:
mapdl.parameters.keys()

## Saving 3D Object

In [1]:
import os

current_dir = os.getcwd()

In [19]:
mapdl.cdwrite(fname=f"{current_dir}/result/Kagome_3", ext = "txt")

*** MAPDL GLOBAL STATUS *** 

 TITLE =                                                                       
       504 KEYPOINTS DEFINED     MAX KEYPOINT NUMBER =            504
       756 LINES DEFINED         MAX LINE NUMBER =                756
       254 AREAS DEFINED         MAX AREA NUMBER =                254
         1 VOLUMES DEFINED       MAX VOLUME NUMBER =                1
         0 ELEM TYPES DEFINED    MAX ELEM TYPE NUMBER =             0
         0 ELEMENTS DEFINED      MAX ELEMENT NUMBER =               0
         0 NODES DEFINED         MAX NODE NUMBER =                  0

 WRITE ANSYS DATABASE AS AN ANSYS INPUT FILE: /home/mjk32/IIAD_pyMAPDL/result/Kagome_3.txt                                                                                                                                                                                                                        

 WRITE IGES FILE= /home/mjk32/IIAD_pyMAPDL/result/Kagome_3.iges

 ATTRIBUTES WILL BE WRITTE

In [20]:
mapdl.exit()

In [4]:
result = pymapdl_reader.read_binary(f"{current_dir}/result/temp_triangular.rst")

FileNotFoundError: /home/mjk32/IIAD_pyMAPDL/result/temp_triangular.rst is not a file or cannot be found